In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version 仅存在于 Colab
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pathlib
import numpy as np
import math
from keras_preprocessing import image


In [ ]:
import pandas as pd
train_data=pd.read_csv("word2.csv")
all_img_path=train_data['PATH']
all_label=train_data['VALUE']


# Select the first 30000 captions from the shuffled set
num_examples = 10000
num_start=0
all_img_path = all_img_path[num_start:].values
all_label = all_label.values[num_start:]
all_label = list('@'+l+'^' for l in all_label)
all_img_path = list(l+"" for l in all_img_path)

from sklearn.utils import shuffle
all_img_path,all_label=shuffle(all_img_path,all_label)
all_img_path=all_img_path[:num_examples]
all_label=all_label[:num_examples]
print(len(all_label))
print(all_label[:15])


In [ ]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters=' ',
      oov_token="<unk>",
      char_level=True,
      lower=False)
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)

    # lang_tokenizer.word_index['<pad>'] = 0
    # lang_tokenizer.index_word[0] = '<pad>'
    tensor = lang_tokenizer.texts_to_sequences(lang)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post')
    #max_length = max(len(t) for t in tensor)

    return tensor, lang_tokenizer#, max_length

In [ ]:
sample_label,sample_lang=tokenize(all_label[1])
print(sample_label.shape)
print(sample_lang)
#print(sample_length)

all_label_tensor,all_label_lang=tokenize(all_label)
def convert(lang, tensor):
    for t in tensor:
        if t!=0:
            print ("%d ----> %s" % (t, lang.index_word[t]))
convert(all_label_lang,all_label_tensor[1])
# tf.expand_dims([all_label_lang.word_index['<unk>']] * 64, 1)
all_label_lang.index_word


In [ ]:
BUFFER_SIZE = len(all_img_path)
BATCH_SIZE = 256
steps_per_epoch = len(all_img_path)//BATCH_SIZE
units = 256
vocab_tar_size = len(all_label_lang.word_index)+1
embedding_dim = 3 #embedding_dimensions =  number_of_categories**0.25
max_length=len(all_label_tensor[0])
features_shape = 2048
attention_features_shape = 91
print(max_length)
print(vocab_tar_size)
img_height=48
img_width=192

In [ ]:
img_name_train, cap_train=all_img_path,all_label_tensor

In [ ]:
# Create training and validation sets using an 80-20 split
# img_name_train, img_name_val, cap_train, cap_val = train_test_split(all_img_path,
#                                     all_label_tensor, test_size=0.2, random_state=0)

In [ ]:
def detect_contour(img):
    size=img.shape
    # print(size)
    x=0
    while not tf.reduce_any(tf.not_equal(img[:,x],1)):
      x+=5
    xmin=x-5 if x-5>0 else 0
    x=size[1]-1
    while not tf.reduce_any(tf.not_equal(img[:,x],1)):
      x-=5
    xmax=x+5 if x+5<size[1]-1 else size[1]-1

    return 0,xmin,size[0],xmax

In [ ]:
import cv2
def slide_window(img):
    xstep=0
    img_patches=[]
    while xstep < img_width-10:
        img_patches.append(img[:,xstep:xstep+10])
        xstep+=2
    return img_patches

def precess_image(image_path):

    img = slant_correct(image_path)
    size = img.shape
    ymin,xmin,ymax,xmax=[0,0,size[0],size[1]]
    # width=0
    # img=tf.convert_to_tensor(img)
    if (ymax-ymin)<img_height:
        width=round((xmax-xmin)*(img_height/(ymax-ymin)))
    else:
        width=round((xmax-xmin)/((ymax-ymin)/img_height))
    if width>img_width:
        img = cv2.resize(img.numpy(),(img_width,img_height))
        width=img_width
    else:
        img = cv2.resize(img.numpy(),(width,img_height))
    
    img=tf.pad(img,[[0,0],[0,img_width-width]],constant_values=1)
    img_patches=slide_window(img)
    return img_patches

def show_patches(img):
    fig = plt.figure(figsize=(10, 10))
    i=0
    #sample_image=tf.reshape(sample_image,(y,x))
    for i in range(len(img)):
        ax = fig.add_subplot(10, 10, i+1)
        ax.set_title(str(i))
        ax.imshow(img[i],cmap='gray')
    plt.tight_layout()
    plt.show()

def slant_correct(file_path):
    img = cv2.imread(file_path)#,cv2.IMREAD_GRAYSCALE)
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    ret, binary = cv2.threshold(gray,230,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    binary = 255 - binary
    contours, hierarchy = cv2.findContours(binary,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

    sum_theta = 0
    sum_weight = 0
    for line in range(len(contours)):

        cv2.drawContours(img,contours[line],-1,(0,0,255),3) 

        columns = []

        for i in range(1,len(contours[line])):
            columns.append(contours[line][i]-contours[line][i - 1])
        columns.append(contours[line][0]-contours[line][-1])

        a = []

        for i in range(len(columns)):
            if columns[i][0][0] == 0 and columns[i][0][1] == -1:
                a.append(6)
            elif columns[i][0][0] == 0 and columns[i][0][1] == 1:
                a.append(2)
            elif columns[i][0][0] == 1 and columns[i][0][1] == 1:
                a.append(1)
            elif columns[i][0][0] == 1 and columns[i][0][1] == 0:
                a.append(0)
            elif columns[i][0][0] == 1 and columns[i][0][1] == -1:
                a.append(7)
            elif columns[i][0][0] == -1 and columns[i][0][1] == 1:
                a.append(3)
            elif columns[i][0][0] == -1 and columns[i][0][1] == 0:
                a.append(4)
            elif columns[i][0][0] == -1 and columns[i][0][1] == -1:
                a.append(5)

        THRESH_V=1
        status=np.zeros_like(a)

        v_up = [2, 1, 0, 1, 2, 3, 4, 3]
        v_down = [2, 3, 4, 3, 2, 1, 0, 1]
        for i in range(len(a)):
            sum_slope_up = 0
            sum_slope_down = 0
            array=[]
            array.append(i-2)
            array.append(i-1)
            array.append(i)
            array.append(i+1 if i+1<len(a) else i+1-len(a))
            array.append(i+2 if i+2<len(a) else i+2-len(a))
            for j in array:
                sum_slope_up = sum_slope_up + v_up[a[j]]
                sum_slope_down = sum_slope_down + v_down[a[j]]

            if sum_slope_up < THRESH_V:
                status[i] = 1
            if sum_slope_down < THRESH_V:
                status[i] = -1
        cv2.waitKey(0)
#         print(status)
        p=contours[line][np.where(status==0)]

        try:
            for k in range(len(p)-1):

                dirt=p[k][0]-p[k+1][0]
                theta=-1*(dirt[1]/dirt[0])
                if theta>=0:
                    theta=theta if theta<1 else 1.
                else:
                    theta=theta if theta>-1 else -1.
                
                sum_theta+=theta*abs(dirt[1])
                sum_weight+=abs(dirt[1])

        except:
            pass
        
    h,w = gray.shape
    res = gray
    if sum_weight!=0:
      M=np.array([[1,sum_theta/sum_weight,0.5*w*abs(sum_theta/sum_weight)],[0,1,0]])
      res = cv2.warpAffine(gray, M,(w*2, h),flags=cv2.INTER_CUBIC,borderValue=(255))
      res=tf.cast(res,dtype=tf.float32)/255.
      res=tf.where(res>0.88,tf.ones_like(res),res)
      try:
        ymin,xmin,ymax,xmax = detect_contour(res)
        return res[ymin:ymax,xmin:xmax]
      except:
        return res
    else:
      res=tf.cast(res,dtype=tf.float32)/255.
      res=tf.where(res>0.88,tf.ones_like(res),res)
      return res

In [ ]:
i=np.random.randint(0,len(all_img_path),1)[0]
# img=precess_image("words/m04/m04-043/m04-043-00-08.png")
img=precess_image(all_img_path[i])
# words/b06/b06-082/b06-082-09-02.png
# words/m04/m04-043/m04-043-00-08.png
# words/g06/g06-031c/g06-031c-01-04.png
# plt.imshow(img[5],cmap='gray')
show_patches(img)
print(all_label[i])
img=tf.convert_to_tensor(img)
print(img.shape)

In [ ]:
# Load the numpy files
def map_func(img_name, cap):
    name = img_name.decode('utf-8')
    img_tensor = tf.convert_to_tensor(precess_image(name))
    return img_tensor, cap

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((img_name_train, cap_train))
# dataset = tf.data.Dataset.from_tensor_slices((all_img_path, all_label_tensor))
dataset = dataset.shuffle(BUFFER_SIZE)

# Use map to load the numpy files in parallel
dataset = dataset.map(lambda item1, item2: tf.numpy_function(# 包上np函数作为tf函数
          map_func, [item1, item2], [tf.float32, tf.int32]),
          num_parallel_calls=tf.data.experimental.AUTOTUNE)

dataset = dataset.batch(BATCH_SIZE,drop_remainder=True)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, enc_units,batch_sz):
        super(Encoder, self).__init__()
        self.enc_units = enc_units
        self.conv1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(20,5,strides=1,
                                          activation='relu',
                                          padding='same',
                                          kernel_initializer='he_uniform'))
        self.maxpool1 = tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPool2D((2,2)))
        self.conv2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(50,5,strides=1,
                                            activation='relu',
                                            padding='same',
                                            kernel_initializer='he_uniform'))
        self.maxpool2 = tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPool2D((2,2)))
        self.flatten = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())
        self.dense = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1024))
        self.dropout1 = tf.keras.layers.Dropout(0.5)
        
        self.LSTM = tf.keras.layers.LSTM(self.enc_units,  
                                         return_sequences=True)
        self.BLSTM = tf.keras.layers.Bidirectional(self.LSTM)
        self.dropout2 = tf.keras.layers.Dropout(0.5)
        self.LSTM2 = tf.keras.layers.LSTM(self.enc_units,
                                       return_sequences=True,
                                       return_state=True)
        self.BLSTM2=tf.keras.layers.Bidirectional(self.LSTM2)

    def call(self, img, hidden):
        x=tf.expand_dims(img,-1)
        # print(batch_seq.shape)
        print(x.shape)
        x=self.conv1(x)
        x=self.maxpool1(x)
        x=self.conv2(x)
        x=self.maxpool2(x) 
        x=self.flatten(x)
        x=self.dense(x)
        x=self.dropout1(x)
        output = self.BLSTM(x)#h短期记忆,c长期记忆
        output = self.dropout2(output)
        # output = self.BLSTM3(output)
        output, state_h, state_c, b_state_h, b_state_c = self.BLSTM2(output)
        # output, state_h, state_c = self.BLSTM2(output)
        return output, [tf.concat([state_h,b_state_h],axis=-1),tf.concat([state_c,b_state_c],axis=-1)]
        # return output, [state_h,state_c]


    def initialize_hidden_state(self,batch_sz):
        return tf.zeros((batch_sz, self.enc_units))

In [ ]:
encoder = Encoder(units,BATCH_SIZE)
# image_dataset=tf.data.Dataset.from_generator(load_image,(tf.float32,tf.string)).batch(BATCH_SIZE)
hidden=encoder.initialize_hidden_state(BATCH_SIZE)

sample_output,sample_hidden=encoder(tf.expand_dims(precess_image(all_img_path[1]), 0),hidden)
print("sample_output shape: (batch size, sequence_length, units) {}".format(sample_output.shape))
print("sample_hidden shape: (1, units) {}".format(sample_hidden[1].shape))

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        # 隐藏层的形状 == （批大小，隐藏层大小）
        # hidden_with_time_axis 的形状 == （批大小，1，隐藏层大小）
        # 这样做是为了执行加法以计算分数  
        hidden_with_time_axis = tf.expand_dims(hidden, 1)

        # 分数的形状 == （批大小，最大长度，1）
        # 我们在最后一个轴上得到 1， 因为我们把分数应用于 self.V
        # 在应用 self.V 之前，张量的形状是（批大小，最大长度，单位）
        score = self.V(tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis)))

        # 注意力权重 （attention_weights） 的形状 == （批大小，最大长度，1）
        attention_weights = tf.nn.softmax(score, axis=1)

        # 上下文向量 （context_vector） 求和之后的形状 == （批大小，隐藏层大小）
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [ ]:
attention_layer = BahdanauAttention(units)
attention_result, attention_weights = attention_layer(sample_output,sample_hidden[1])

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units):
        super(Decoder, self).__init__()
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(self.dec_units, 
                                       return_sequences=True,
                                       return_state=True)
                                      
        self.fc1 = tf.keras.layers.Dense(vocab_size)

        # 用于注意力
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, enc_output, hidden):
        # 编码器输出 （enc_output） 的形状 == （批大小，最大长度，隐藏层大小）
        context_vector, attention_weights = self.attention(enc_output, hidden[1])
        # x 在通过嵌入层后的形状 == （批大小，1，嵌入维度）
        x = self.embedding(x)

        # x 在拼接 （concatenation） 后的形状 == （批大小，1，嵌入维度 + 隐藏层大小）
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # 将合并后的向量传送到 GRU
        output, state_h, state_c = self.lstm(x,hidden)

        output = tf.concat([tf.expand_dims(context_vector, 1), output], axis=-1)

        output = self.fc1(output)
        # 输出的形状 == （批大小 * 1，隐藏层大小）
        output = tf.reshape(output, (-1, output.shape[2]))

        return output, [state_h,state_c], attention_weights

In [ ]:
decoder = Decoder(vocab_tar_size, embedding_dim, units*2)

sample_decoder_output, _, _ = decoder(tf.random.uniform((1, 1)), sample_output, sample_hidden)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001,decay=0.02)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
acc_object=tf.keras.metrics.SparseCategoricalAccuracy()

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  # loss_ = loss_object(real,tf.clip_by_value(pred,1e-15,1.0))
  loss_ = loss_object(real,pred)
  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

def acc_function(real,pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  acc_ = acc_object(real, pred)
  mask = tf.cast(mask, dtype=acc_.dtype)
  acc_ *= mask

  return tf.reduce_mean(acc_)

In [ ]:
import os
checkpoint_path = './training_checkpoints/81_train'
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=2)

In [ ]:
start_epoch = 0
if ckpt_manager.latest_checkpoint:
  start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
  # restoring the latest checkpoint in checkpoint_path
  ckpt.restore(ckpt_manager.latest_checkpoint)

In [ ]:
# adding this in a separate cell because if you run the training cell
# many times, the loss_plot array will be reset
loss_plot = []

In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0
    acc=0
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        # tf.print(enc_hidden)
        dec_input = tf.expand_dims([all_label_lang.word_index['@']]*inp.shape[0], 1)
        # print(dec_input)
        # 教师强制 - 将目标词作为下一个输入
        for t in range(1, targ.shape[1]):
            # 将编码器输出 （enc_output） 传送至解码器
            predictions, dec_hidden, _ = decoder(dec_input, enc_output, dec_hidden)
            loss += loss_function(targ[:, t], predictions)
            acc += acc_function(targ[:,t],predictions)
            # 使用教师强制
            # print(targ[:,t])
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))
    acc=acc/int(targ.shape[1])
    #print(batch_loss.numpy())
    variables = encoder.trainable_variables + decoder.trainable_variables
    
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return loss,batch_loss,acc

In [ ]:
@tf.function
def vad_step(inp, targ,enc_hidden):

    enc_output, enc_hidden = encoder(inp,enc_hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([all_label_lang.word_index['@']]*inp.shape[0], 1)

    acc=0
    for t in range(1, targ.shape[1]):
      predictions, dec_hidden, _ = decoder(dec_input, enc_output, dec_hidden)
      
      predicted_id = tf.argmax(predictions, axis=-1)
      acc += acc_function(targ[:,t],predictions)
      dec_input = tf.expand_dims(predicted_id, 1)

    return acc/ int(targ.shape[1])

In [ ]:
start_epoch=0

In [ ]:
EPOCHS = 10
import time
for epoch in range(start_epoch,EPOCHS):
    start = time.time()
    total_loss = 0
    total_acc = 0
    
    for (batch,(inp,targ)) in enumerate(dataset.take(steps_per_epoch)):
        enc_hidden=encoder.initialize_hidden_state(BATCH_SIZE)
        # print(targ)
        t_loss,batch_loss,acc = train_step(inp, targ,enc_hidden)
        total_loss += batch_loss
        total_acc+=acc
        if batch % 2 == 0:
            # print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss.numpy()))
            print('Epoch {} Batch {} Loss {:.4f} acc {:.4f}'.format(epoch + 1, batch, batch_loss.numpy(),acc.numpy()))
            
    loss_plot.append(total_loss / steps_per_epoch)
    
    # total_acc = vad_step()
    # validsize=round(steps_per_epoch*0.2)
    # for (vbatch,(vinp,vtarg)) in enumerate(dataset.skip(np.random.randint(0,steps_per_epoch-validsize,size=1)[0]).take(validsize)):
    #   batch_acc = vad_step(vinp, vtarg,enc_hidden)
    #   total_acc += batch_acc

      
    # 每 2 个周期（epoch），保存（检查点）一次模型
    # if (epoch + 1) % 2 == 0:
    # ckpt_manager.save()
    
    print('Epoch {} Loss {:.4f} Acc {:.4f}'.format(epoch + 1, total_loss / steps_per_epoch, total_acc / steps_per_epoch))
    # print('Epoch {} Loss {:.4f} '.format(epoch + 1, total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [ ]:
def evaluate(image):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = encoder.initialize_hidden_state(1)
    temp_input = tf.expand_dims(tf.convert_to_tensor(precess_image(image)), 0)
    # print(temp_input.shape)
    features,hidden = encoder(temp_input,hidden)

    dec_input = tf.expand_dims([all_label_lang.word_index['@']], 0)
    # print(dec_input)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.argmax(predictions, 1).numpy()[0]
        # predicted_id=tf.random.categorical(predictions,1)[0][0].numpy()
        print(predicted_id)
        result.append(all_label_lang.index_word[predicted_id])

        if all_label_lang.index_word[predicted_id] == '^':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot

In [ ]:
from glob import glob
from PIL import Image
import pickle
def plot_attention(image, result, attention_plot):
    img = cv2.imread(image,cv2.IMREAD_GRAYSCALE)
    size = img.shape
    ymin,xmin,ymax,xmax=[0,0,size[0],size[1]]
    if (ymax-ymin)<img_height:
        width=round((xmax-xmin)*(img_height/(ymax-ymin)))
    else:
        width=round((xmax-xmin)/((ymax-ymin)/img_height))
    if width>img_width:
        img = cv2.resize(img,(img_width,img_height),cv2.INTER_AREA)
        width=img_width
    else:
        img = cv2.resize(img,(width,img_height),cv2.INTER_AREA)
    img = tf.cast(img,tf.float32)/255.
    img=tf.where(img>0.88,tf.ones_like(img),img)
    timg=tf.pad(img,[[0,0],[0,img_width-width]],constant_values=1)
    fig = plt.figure(figsize=(100, 100))

    len_result = len(result)
    print(len_result)
    for l in range(len_result):
        temp_att = attention_plot[l]
        ax = fig.add_subplot(len_result+1//2, len_result+1//2, l+1)
        ax.set_title(result[l])
        
        img = ax.imshow(timg,cmap='gray')
        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

    plt.tight_layout()
    plt.show()

In [ ]:
# captions on the validation set
rid = np.random.randint(0, len(all_img_path))
image = all_img_path[rid]
print(image)
real_caption=all_label[rid]
result, attention_plot = evaluate(image)

print ('Real Caption:', real_caption)
print ('Prediction Caption:', ' '.join(result))
plot_attention(image, result, attention_plot)


In [ ]:
p="words/b01/b01-023/b01-023-00-04.png"
' '.join(evaluate(p)[0])